In [1]:
from cite_demo.cite_modules.LLM import LLM
from cite_demo.cite_modules.augment_model import AttributingModule
from cite_demo.pipeline.pipeline import Sequence
from cite_demo.prompt.prompt import Prompt
from cite_demo.Dataset.Dataset import FileDataset
from cite_demo.evaluator.evaluator import DefaultEvaluator
import json

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shenjiajun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/shenjiajun/miniconda3/envs/torch_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Loading

In [2]:
dataset = FileDataset('data/asqa.json')

with open('prompts/asqa.json','r',encoding='utf-8') as file:
        demo = json.load(file)
        instruction =  demo['INST']
print(instruction)

Instruction: Write an accurate, engaging, and concise answer for the given question using only the provided search results (some of which might be irrelevant) and cite them properly. Use an unbiased and journalistic tone. Always cite for any factual claim. When citing several search results, use [1][2][3]. Cite at least one document and at most three documents in each sentence. If multiple documents support the sentence, only cite a minimum sufficient subset of the documents.


# Make a Request Wrapper

The request wrapper contains is a template that consist of different components. Components with no data passing in will be automatically dropped.

In [3]:
prompt = Prompt(template='<INST><question><docs><prefix><span>Answer: ',
                components={'INST':'{INST}\n\n', 
                            'question':'Question:{question}\n\n',
                            'docs':'{docs}\n',
                            'span':'The highlighted spans are: \n{span}\n\n',
                            'prefix':'Prefix: {prefix}\n\n',
                            })

print(prompt)

{INST}

Question:{question}

{docs}
Prefix: {prefix}

The highlighted spans are: 
{span}

Answer: 


# Define an Evaluater
An Evaluator will automatically evaluate the output, once some metrics are defined. You can use some pre-defined ones or new ones.

In [4]:
evaluator = DefaultEvaluator(criteria = ['str_em','length','rouge'])

# Define Generation Modules and Enhancing Modules

In [5]:
# attributer = AttributingModule(model='gpt-3.5-turbo')
llm = LLM(model='gpt-3.5-turbo',prompt_maker=prompt, self_prompt={'INST':instruction})

# Construct a Pipeline

In [ ]:
pipeline = Sequence(sequence = [llm], head_prompt_maker = prompt, evaluator = evaluator, dataset = dataset)

# Run

In [ ]:
pipeline.run_on_dataset(datakeys=['question','docs'], init_docs='docs')